<a href="https://colab.research.google.com/github/Siddharthg97/Text-generation-comments/blob/main/Supervised_finetuning_for_text_completion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install accelerate -U

In [ ]:
Here we perform supervise fine tuning for text completion using GPT2LMHeadModel & T5ForConditionalGeneration.
We apped the the prompt and comment while training and in testing we keep only prompt.Specify the length of text generation as
200.
Now we notice that comments of analyst not only contains anchors used for discounting about also some other information in form
of narration like watch-list is a also customer and then start giving description of information of account and transactions.
, can got other sources like host,EBPP ,APEX etc.
So they use to perform an exercise.
2)Further they may give information about parties related to watchlist which has nothing to with discounting.Any
surplus information.Information/KYC of other parties
3) Giving info of irrelevant sources, miscellanous anchors that were not useful in discounting etc.

In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer,T5ForConditionalGeneration,T5Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os
import re
from transformers import TextDataset
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [2]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import pandas as pd
import numpy as np
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import ast
pd.set_option("Display.max_colwidth",None)

In [4]:
device = 'cpu'


In [3]:
# l_samp=pd.read_csv("/content/complete_prompt.csv")
l_samp=pd.read_csv("/content/sample_columns_prompt_with_dob_name_data (1).csv")

In [ ]:
l_samp.shape
l_samp.columns

In [6]:
l_samp["prompt_&_comment"]

0                The customer and watch person date of birth is 1982/03/13 and 1982/03/05 respectively, with their date of birth similarity score as 0.8.Their names are ZHANGA TING and BING ZHANG respectively, with  their string based name matching scores as 7.588658000601267e-12, 90.0 and sound based name matching score as 0.6.Thus, customer and watchlist are a mis-match.Mismatch Full Legal Name_Name Does Not Sound Similar
1                        The customer and watch person date of birth is 1993/12/16 and  respectively, with their date of birth similarity score as -1.0.Their names are TRAN THI HA and THI HOA TRAN respectively, with  their string based name matching scores as 5.03639903115527e-08, 92.0 and sound based name matching score as 0.5.Thus, customer and watchlist are a mis-match.Mismatch Full Legal Name_Name Does Not Sound Similar
2                          The customer and watch person date of birth is 1982/03/13 and  respectively, with their date of birth similarity scor

In [ ]:
### Train & Test dataset

In [5]:
#Create a very small test set to compare generated text with the reality
l_test= l_samp.sample(100)
l_train = l_samp.loc[~l_samp.index.isin(l_test.index)]

#Reset the indexes
l_test = l_test.reset_index()


In [6]:
l_train.shape,l_samp.shape,l_test.shape

((64227, 4), (64327, 4), (100, 5))

GPT2 Model

In [ ]:
package="gpt2"
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
model=GPT2LMHeadModel.from_pretrained("gpt2")

In [12]:
l_train["prompt_&_comment"].to_csv("Train_dataset.csv")

In [13]:


train_path = "Train_dataset.csv"
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=1024  # Adjust as needed
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [14]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [15]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
print(device)

cpu


In [16]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)
trainer.train()
model.save_pretrained("./gpt2-finetuned")

Step,Training Loss


KeyboardInterrupt: 

In [16]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=50, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
              generated_list.append(output_text)

    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_comments = []
  for i in range(2):
    x = generate(model.to("cpu"), tokenizer, test_data["prompt"][i], entry_count=1)
    generated_comments.append(x)

  return generated_comments

In [21]:
l_test.columns

Index(['index', 'Unnamed: 0', 'prompt', 'comments', 'prompt_&_comment',
       'comments_gen'],
      dtype='object')

In [34]:
l_test_2=l_test[:2]

In [35]:
l_test_2

,index,Unnamed: 0,prompt,comments,prompt_&_comment,comments_gen
0,42172,42172,"The customer date of birth and watch person year of birth is 1980/09/11['1964', '1965', '1966', '1967', '1968'] respectively,with their date of birth similarity score as 0.0.Their names are MUHAIMI BIN HARUN and SUHAIMI HARUN respectively, with their string based name matching scores as 2.917718289197068e-09, 93.0 and sound based name matching score as 0.6333333333333333.Thus, customer and watchlist are a mis-match.",Mismatch Year of Birth (YOB),"The customer date of birth and watch person year of birth is 1980/09/11['1964', '1965', '1966', '1967', '1968'] respectively,with their date of birth similarity score as 0.0.Their names are MUHAIMI BIN HARUN and SUHAIMI HARUN respectively, with their string based name matching scores as 2.917718289197068e-09, 93.0 and sound based name matching score as 0.6333333333333333.Thus, customer and watchlist are a mis-match.Mismatch Year of Birth (YOB)",
1,10474,10474,"The customer and watch person date of birth is 1984/10/24 and respectively, with their date of birth similarity score as -1.0.Their names are MUHAMMAD ASIM and MUHAMMAD QASIM respectively, with their string based name matching scores as 6.294637255876978e-09, 90.0 and sound based name matching score as 0.8846153846153846.Thus, customer and watchlist are a mis-match.",Mismatch Full Legal Name_Name Does Not Sound Similar,"The customer and watch person date of birth is 1984/10/24 and respectively, with their date of birth similarity score as -1.0.Their names are MUHAMMAD ASIM and MUHAMMAD QASIM respectively, with their string based name matching scores as 6.294637255876978e-09, 90.0 and sound based name matching score as 0.8846153846153846.Thus, customer and watchlist are a mis-match.Mismatch Full Legal Name_Name Does Not Sound Similar",


In [36]:
#Run the functions to generate the
l_test_2["comments_gen"]=""
l_test_2["comments_gen"].loc[:1]=text_generation(l_test_2)

<ipython-input-36-201263e2d242>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_test_2["comments_gen"]=""
100%|██████████| 1/1 [00:25<00:00, 25.82s/it]
<ipython-input-36-201263e2d242>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_test_2["comments_gen"].loc[:1]=text_generation(l_test_2)


In [37]:
l_test_2["comments_gen"].loc[:1]
# l_test["comments_gen"].apply(lambda x: x.split("match.",1)[1])[0]

0    [The customer date of birth  and watch person  year of birth is 1980/09/11['1964', '1965', '1966', '1967', '1968'] respectively,with their date of birth similarity score as 0.0.Their names are MUHAIMI BIN HARUN and SUHAIMI HARUN respectively, with  their string based name matching scores as 2.917718289197068e-09, 93.0 and sound based name matching score as 0.6333333333333333.Thus, customer and watchlist are a mis-match.Mismatch Year of Birth (YOB)"\n41492,"The customer date of birth  and watch person  year of birth is 1954/10/03['1964', '1965', '1966', '1967', '1968'] respectively,<|endoftext|>]
1                                                                             [The customer and watch person date of birth is 1984/10/24 and  respectively, with their date of birth similarity score as -1.0.Their names are MUHAMMAD ASIM and MUHAMMAD QASIM respectively, with  their string based name matching scores as 6.294637255876978e-09, 90.0 and sound based name matching score as 0.88461

In [ ]:
l_test["Only_comments_gen"]=l_test["comments_gen"].apply(lambda x: x.split("match.",1)[1].rstrip('<|endoftext|>'))

T5 model

In [7]:
from transformers import T5Tokenizer,T5ForConditionalGeneration


In [8]:
tokenizer=T5Tokenizer.from_pretrained("t5-base")
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

In [9]:
training_args_t5 = TrainingArguments(
    output_dir="./t5-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)



In [17]:
trainer_t5 = Trainer(
    model=model_t5,
    args=training_args_t5,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [1]:
trainer_t5.train()
model.save_pretrained("./t5-finetuned")

NameError: name 'trainer_t5' is not defined

In [ ]:
#Run the functions to generate the
l_test["comments_gen"]=""
l_test["comments_gen"].loc[:1]=text_generation(l_test)

In [ ]:

# input_text = l_test["prompt"].loc[0]
# input_ids = tokenizer.encode(input_text,max_length=900,Truncation=True ,return_tensors="pt")

# output = model.generate(input_ids, max_length=100, num_return_sequences=1,early_stopping=True,num_beams=5,
#           early_stopping=True,pad_token_id=tokenizer.eos_token_id)


# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print("Generated text:", generated_text)

In [ ]:
outputs